In [1]:
import numpy as np 
import pandas as pd 

import torch 

In [2]:
train_data = pd.read_csv('/kaggle/input/eedi-mining-misconceptions-in-mathematics/train.csv')
misconception = pd.read_csv('/kaggle/input/eedi-mining-misconceptions-in-mathematics/misconception_mapping.csv')
test_data  = pd.read_csv('/kaggle/input/eedi-mining-misconceptions-in-mathematics/test.csv')



In [3]:
''' 

SO trian data - has a question(simple maths )
and answers are - A,B ,C ,D out of which ONly 1 is correct . 

Now , if opttion D is correct , other option (A ,B,C) wrong ,  
So we are also given reaseon for them being wrong( using misconception id no. from 
misconception dataframe ). 

now ,we need to predict this misconception id for test data . 

e.g. Do 2 x3 - 5 . 

A. 1 B. 0 C. 9 D. 2 
correct ans =A 
wrong ans = B ,C ,D 

msconceptionA - nan , misconcB  -23 id no (wronf rules) , misconcepC (wrong info) 

Predict misconception wrt a question 

'''

' \n\nSO trian data - has a question(simple maths )\nand answers are - A,B ,C ,D out of which ONly 1 is correct . \n\nNow , if opttion D is correct , other option (A ,B,C) wrong ,  \nSo we are also given reaseon for them being wrong( using misconception id no. from \nmisconception dataframe ). \n\nnow ,we need to predict this misconception id for test data . \n\ne.g. Do 2 x3 - 5 . \n\nA. 1 B. 0 C. 9 D. 2 \ncorrect ans =A \nwrong ans = B ,C ,D \n\nmsconceptionA - nan , misconcB  -23 id no (wronf rules) , misconcepC (wrong info) \n\nPredict misconception wrt a question \n\n'

In [4]:
train_data.iloc[1]

QuestionId                                                          1
ConstructId                                                      1612
ConstructName       Simplify an algebraic fraction by factorising ...
SubjectId                                                        1077
SubjectName                           Simplifying Algebraic Fractions
CorrectAnswer                                                       D
QuestionText        Simplify the following, if possible: \( \frac{...
AnswerAText                                                 \( m+1 \)
AnswerBText                                                 \( m+2 \)
AnswerCText                                                 \( m-1 \)
AnswerDText                                         Does not simplify
MisconceptionAId                                               2142.0
MisconceptionBId                                                143.0
MisconceptionCId                                               2142.0
MisconceptionDId    

In [5]:
index =1 



def dataset_load( index ) :
    

    idea = train_data.iloc[index]['ConstructName']
    quesn = train_data.iloc[index]['QuestionText']
    ans_opt = train_data.iloc[index]['CorrectAnswer']
    ans = ('Answer' + ans_opt + 'Text')
    ans = train_data.iloc[index][ans]

    prompt1  = f'The basic idea of this question is to {idea}.Now ,the actual question is to - {quesn}.'

    prompt2 =f'now the correct answer is- Option{ans_opt},that is {ans}.'

    prompt3  = f'The rest of the answers are wrong. And the reason for this misconception are-'


    options = ['A' ,'B' ,'C' ,'D'] 

    quesn_prompt = prompt1
    ans_prompt =  prompt2 + prompt3 

    full_ans_str = ''
    
    Misc_ID = []
    
    for opt in options :
        if opt is not np.nan :
            misc = ('Misconception'+ opt+ 'Id')
            misc_value = train_data.iloc[index][misc]
            
            # Check if the misconception ID is not NaN
            if pd.notna(misc_value):
                misc_id = int(misc_value)
                misc_text = misconception.iloc[misc_id][1]

                prompt_ = f'Option {opt} is wrong because {misc_text}.'
                ans_prompt += prompt_ + f'hence , the misconception id are-{misc_id}\n'
                full_ans_str+=ans_prompt
                Misc_ID.append( str(misc_id) )

                
    quesn_str = '<s>[INST]' + quesn_prompt + '[/INST]'
    misc_ = ' '.join(Misc_ID)
    full_ans_str_ = f'The Misconception ID are {misc_}\n'
    final_ans = full_ans_str_ + full_ans_str
    
    Question.append(quesn_str)
    Misconception_Id.append( final_ans )
           
        
        

In [6]:
Question = []
Misconception_Id = []


for index in range( 50 ) :
    dataset_load(index)

/tmp/ipykernel_30/4197465484.py:38: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  misc_text = misconception.iloc[misc_id][1]


In [7]:
data = { 'Question' :Question , 'Misconception_Id': Misconception_Id }

df = pd.DataFrame( data)

df.iloc[1][0]


/tmp/ipykernel_30/1465143998.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  df.iloc[1][0]


'<s>[INST]The basic idea of this question is to Simplify an algebraic fraction by factorising the numerator.Now ,the actual question is to - Simplify the following, if possible: \\( \\frac{m^{2}+2 m-3}{m-3} \\).[/INST]'

In [8]:
''' Think how to prepare datset '''
Text = []

for i in range( df.shape[0]) :
    seg = df.iloc[i][0] + df.iloc[i][1]
    Text.append( seg )

data ={ 'text' : Text }

data1 = pd.DataFrame( data)
data1.head(3)

/tmp/ipykernel_30/3545490196.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  seg = df.iloc[i][0] + df.iloc[i][1]


,text
0,<s>[INST]The basic idea of this question is to...
1,<s>[INST]The basic idea of this question is to...
2,<s>[INST]The basic idea of this question is to...


In [9]:
# !pip install datasets 

from datasets import Dataset, DatasetDict

data_new  = Dataset.from_pandas(data1)
#data_new.push_to_hub("misconception-llama2-1k")
# data_new.save_to_disk("data_new.hf")

In [10]:
# !pip install -q accelerate peft bitsandbytes transformers trl


In [11]:
# Importing important Libraries 
# peft = Paramter efficient fine tuning of Large language models (contans Lora method ) 
# to save GPU
# bitsandbytes for Quantization 

!pip install -q accelerate peft bitsandbytes transformers trl

# import other transformer imp. modules 
import os 
import torch 

torch.cuda.empty_cache()

# dAtaset 
from datasets import load_dataset 

from transformers import (
    AutoModelForCausalLM ,
    AutoTokenizer ,
    BitsAndBytesConfig , 
    HfArgumentParser , 
    TrainingArguments , 
    pipeline , 
    logging 
)

# Lora and Transfre learning approaches downloaded 
from peft import LoraConfig , PeftModel 
from trl import SFTTrainer 
# Peft - works liek Transfer learning . 

# while fine tuning a pre-trained model(GPT) on our speciic daatset , freezes some 
# weights and biases(that carry imp info) and trians on specific dataset. 

# laod the model 
model_name = "NousResearch/Llama-2-7b-chat-hf"


# custom datset (Q-A dataset)
# dataset_name = "mlabonne/guanaco-llama2-1k"


# Prepare before hand new model name  
fine_tune_model_name = "LLama-7b-fine-tuned "

# QLora (hypermeters)

# dim of the low rank matrixes that will store the chnages in pre-trained weights . 
lora_rank = 64 

# for scaling in QLora (quantization )
lora_alpha = 16


# to prevent ovefitting , dropout 
lora_dropout = 0.1  

# Quantization Pramas 

# to store weights in int( 4 bits)
use_4bit = True 

# orginal weight data type 
dtype_4bit= "float16"

quant_dtype_4bit = "nf4" # either nf4 or fp4 

# nested quantization 
use_nested_quant  = False 


output_dir = "./results"
# trianing( fien tuning params )

num_epochs =1

fp_16 = False 
bp_16 =False 

# batch size per gpuu for trianing 
batch_size_per_gpu_train = 4 

# batch size per gpuu for eval
batch_size_per_gpu_eval = 4 

gradient_accum_step =1 

#check gradients of loss 
gradient_checkpointing = True 

#maximum grad noemal 
max_grad_norm = 0.3  

learning_rate = 0.1 

# for alyer norm 
weight_decay = 0.001 

#optimizer 
optimizer = "paged_adamw_32bit"

# cosie simi learning 
lr_scheduler_type = "cosine"

#override steps 
max_steps =-1 

#warmpup ratio (re trian frm same point)
warm_up = 0.03 


# now to speed up training 
group_by_length = True 

save_steps= 0 

logging_steps= 10

# Supervised tuning (SFT )

max_seq_length = None 
packing =False 


# laod the doel on gpu 0
device_map = {"": 0 }


# Load dataset 
# dataset = load_dataset ( dataset_name  ,split  ="train")
dataset = data_new 

# dataset = dataset.train_test_split(test_size=0.9)['train']

# LAOD MODEL AND TOKENIZER WITHQLORA HYPER

compute_dtype = getattr( torch , dtype_4bit )

#Quantized model loading 
bnb_config  = BitsAndBytesConfig(
    load_in_4bit = use_4bit , 
    
    bnb_4bit_quant_type = quant_dtype_4bit  , 
    
    bnb_4bit_compute_dtype = dtype_4bit  , 
    
    bnb_4it_sue_doublequant = use_nested_quant 
)

# Laod the base model(Llama) - pre-trianed  trained on books web , to usderstannf language

model = AutoModelForCausalLM.from_pretrained(
    model_name , 
    quantization_config = bnb_config, 
    device_map = device_map ,
)

model.config.use_cache = False 
model.config.pretraining_tp = 1 

''' RUns ONLY on GPU (this code )'''
# Load tokenzier  
# to convert word into word embeddings  

#llama tokeizer 
tokenizer = AutoTokenizer.from_pretrained( model_name , trust_remote_code = True )
tokenizer.pad_token = tokenizer.eos_token # end of speed  token 
tokenizer.padding_size = 'right'

# Laod Lora Config 

peft_config = LoraConfig (
    lora_alpha = lora_alpha , 
    lora_dropout = lora_dropout , 
    r = lora_rank , 
    bias ="none" ,
    task_type= "CAUSAL_LM"
)
# Define trainign arguments of LLama 2 model for fien-tuning 

training_arguments  = TrainingArguments (
    
    output_dir= "./results" , # output folder 
    
    num_train_epochs =  2  , # epochs for trianig 
    
    per_device_train_batch_size = 2 , 
    
    gradient_accumulation_steps  = 2 , 
    
    optim = "paged_adamw_32bit" , 
    
    save_steps = 2 , 
    
    logging_steps = 10 , 
    
    learning_rate = 2e-3 , # keep it low for better converge in backprop 
    
    weight_decay = 0.001 , 
    
    fp16  = True  , 
    bf16 =  False , 
    
    max_grad_norm = 0.3 , 
    
    max_steps = -1 , 
    
    warmup_ratio = 0.03  , 
    
    group_by_length = True , 
    
    lr_scheduler_type = "cosine", 
    
    report_to = "tensorboard"
    
)
# Set SFT trainign arguments 

trainer = SFTTrainer (
    
    model = model , 
    train_dataset = dataset , 
    peft_config = peft_config , 
    
    dataset_text_field = "text" , 
    
    max_seq_length = max_seq_length , 
    
    tokenizer = tokenizer , 
    
    args = training_arguments , 
    
    packing = packing 
)

# # export PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True
# nvidia-smi

import os

# Set the environment variable
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"



Unused kwargs: ['bnb_4it_sue_doublequant']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

...

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:292: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:321: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/50 [00:00<?, ? examples/s]

/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:396: UserWarning: You passed a tokenizer with `padding_side` not equal to `right` to the SFTTrainer. This might lead to some unexpected behaviour due to overflow issues when training a model in half-precision. You might consider adding `tokenizer.padding_side = 'right'` to your code.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


In [12]:
trainer.train()

Step,Training Loss
10,1.503500
20,0.664900


TrainOutput(global_step=24, training_loss=0.9985543588797251, metrics={'train_runtime': 188.5899, 'train_samples_per_second': 0.53, 'train_steps_per_second': 0.127, 'total_flos': 1415866027425792.0, 'train_loss': 0.9985543588797251, 'epoch': 1.92})

In [13]:
i =1 
prompt = 'Solve this question ' + test_data.iloc[i]['QuestionText'] + " where "
    
prompt += 'Option A is ' + test_data.iloc[i]['AnswerAText'] + " and \n"
prompt += 'Option B is ' + test_data.iloc[i]['AnswerBText'] + " and \n"
prompt += 'Option C is ' + test_data.iloc[i]['AnswerCText'] + " and \n"
prompt += 'Option D is ' + test_data.iloc[i]['AnswerDText'] + " and \n"
    
prompt += "Give in Only 1 letter the correct Option like -A or B or C or D."
    
# Generate the option response
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=500)
opt = pipe(f"<s>[INST]{prompt}[/INST]")[0]['generated_text']
    
answer_opt = opt.replace(prompt, "").strip()
# answer_opt = answer_opt[:10]
print(answer_opt)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


<s>[INST][/INST]The Misconception ID are 2212 1756 1756 1756
now the correct answer is- Option A,that is \( m+1 \).The rest of the answers are wrong. And the reason for this misconception are-Option B is wrong because When simplifying fractions, divides both terms by a common denominator.hence, the misconception id are-1756
now the correct answer is- Option A,that is \( m+1 \).The rest of the answers are wrong. And the reason for this misconception are-Option B is wrong because When simplifying fractions, divides both terms by a common denominator.hence, the misconception id are-1756
Option C is wrong because When simplifying fractions, divides both terms by a common denominator.hence, the misconception id are-1756
now the correct answer is- Option A,that is \( m+1 \).The rest of the answers are wrong. And the reason for this misconception are-Option B is wrong because When simplifying fractions, divides both terms by a common denominator.hence, the misconception id are-1756
Option C i

In [14]:
import re 

# 1. Extract misconception IDs (numbers after 'misconception id are-')
misconception_ids = re.findall(r'misconception id are-(\d+)', answer_opt)

# 2. Extract correct option (letter after 'correct answer is- Option')
correct_option = re.search(r'correct answer is- Option ([A-D])', answer_opt).group(1)

# 3. Join misconception IDs into a single string separated by spaces
misconception_ids_str = ' '.join(misconception_ids)

# Output the results
print(f"Misconception IDs: {misconception_ids_str}")
print(f"Correct Option: {correct_option}")

Misconception IDs: 1756 1756 1756 1756 1756 2212
Correct Option: A


In [15]:
Question_id = []
misc_id = []

import re 

for i in range( test_data.shape[0] ):
    # Construct the question prompt
    prompt = 'Solve this question ' + test_data.iloc[i]['QuestionText'] + " where "
    
    prompt += 'Option A is ' + test_data.iloc[i]['AnswerAText'] + " and \n"
    prompt += 'Option B is ' + test_data.iloc[i]['AnswerBText'] + " and \n"
    prompt += 'Option C is ' + test_data.iloc[i]['AnswerCText'] + " and \n"
    prompt += 'Option D is ' + test_data.iloc[i]['AnswerDText'] + " and \n"
    
    prompt += "Give in Only 1 letter the correct Option like -A or B or C or D."
    
    # Generate the option response
    pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=500)
    opt = pipe(f"<s>[INST]{prompt}[/INST]")[0]['generated_text']
    
    answer_opt = opt.replace(prompt, "").strip()
    
    misconception_ids = re.findall(r'misconception id are-(\d+)', answer_opt)

# 2. Extract correct option (letter after 'correct answer is- Option')
    correct_option = re.search(r'correct answer is- Option ([A-D])', answer_opt).group(1)

# 3. Join misconception IDs into a single string separated by spaces
    misconception_ids_str = ' '.join(misconception_ids)


    # Convert QuestionId to string before concatenation
    Question_id.append(str(test_data.iloc[i]['QuestionId']) + '_' + correct_option)
    misc_id.append( misconception_ids_str )
    
    # Misconception ID prompt
#     prompt_ = 'Give Only the Misconception Id for this question ' + test_data.iloc[i]['QuestionText'] + " with respect to "
    
#     misc_ID = []
#     for option in ['A', 'B', 'C', 'D']:
#         # Access the correct AnswerText based on the option
#         if option == 'A':
#             answer_text = test_data.iloc[i]['AnswerAText']
#         elif option == 'B':
#             answer_text = test_data.iloc[i]['AnswerBText']
#         elif option == 'C':
#             answer_text = test_data.iloc[i]['AnswerCText']
#         elif option == 'D':
#             answer_text = test_data.iloc[i]['AnswerDText']
        
#         prompt1 = 'Option ' + option + ' is ' + answer_text + ". "
#         prompt_ += prompt1 + "Give Only the number corresponding to the misconception Id. No text should be given in output, only number separated with spaces."
        
#         # Generate the misconception ID response
#         pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length= 500)
#         _id = pipe(f"<s>[INST]{prompt_}[/INST]")[0]['generated_text']
#         answer_id = _id.replace(prompt_, "").strip()
#         misc_ID.append(str(answer_id))
    
    # Join the misconception IDs with spaces
   


In [16]:
ans = { 'QuestionId_Answer' : Question_id , "MisconceptionId" : misc_id}
ans_df = pd.DataFrame( ans )
ans_df.head()



,QuestionId_Answer,MisconceptionId
0,1869_B,2480 2480 1888 2480
1,1870_B,2162 2162 1122 2162 1122 1122
2,1871_B,1518 1518 1518 1518


In [17]:
ans_df.to_csv("submission.csv", index=False)
print("Submission file created successfully!")

Submission file created successfully!
